In [39]:
import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from titanic_pipeline.config import config
import pandas as pd
import joblib
from evidently.dashboard import Dashboard
from evidently.dashboard.tabs import (
    DataDriftTab,
    DataQualityTab
)
from evidently import ColumnMapping

column_mapping = ColumnMapping(target='survived', prediction='prediction')
model = joblib.load(config.PRODUCTION_MODEL_FILE)

raw_prod = pd.read_csv(config.DATASETS_DIR + 'prod1_data.csv')
target_prod = pd.read_csv(config.DATASETS_DIR + 'prod1_truth.csv')

raw_X_test = pd.read_csv(f"{config.PRODUCTION_MODEL_DATA}/X_test.csv")
raw_y_test = pd.read_csv(f"{config.PRODUCTION_MODEL_DATA}/y_test.csv")

dashboard = Dashboard(tabs=[DataQualityTab(), DataDriftTab()])
dashboard.calculate(raw_prod, raw_X_test, column_mapping=column_mapping)
dashboard.save("../reports/data_quality_drift_raw.html")

/Users/andre.bajorat/tec/.titanic/lib/python3.9/site-packages/scipy/stats/_stats_py.py:6766: RuntimeWarning:

divide by zero encountered in true_divide

/Users/andre.bajorat/tec/.titanic/lib/python3.9/site-packages/scipy/stats/_stats_py.py:6766: RuntimeWarning:

divide by zero encountered in true_divide

/Users/andre.bajorat/tec/.titanic/lib/python3.9/site-packages/scipy/stats/_stats_py.py:6766: RuntimeWarning:

divide by zero encountered in true_divide

/Users/andre.bajorat/tec/.titanic/lib/python3.9/site-packages/scipy/stats/_stats_py.py:6766: RuntimeWarning:

divide by zero encountered in true_divide

/Users/andre.bajorat/tec/.titanic/lib/python3.9/site-packages/scipy/stats/_stats_py.py:6766: RuntimeWarning:

divide by zero encountered in true_divide

/Users/andre.bajorat/tec/.titanic/lib/python3.9/site-packages/scipy/stats/_stats_py.py:6766: RuntimeWarning:

divide by zero encountered in true_divide

/Users/andre.bajorat/tec/.titanic/lib/python3.9/site-packages/scipy/stats/_stats_p

In [40]:
processor = model['preprocessor']
processed_prod = pd.DataFrame(processor.transform(raw_prod.copy()), columns=processor.get_feature_names_out())
processed_train = pd.DataFrame(processor.transform(raw_X_test.copy()), columns=processor.get_feature_names_out())

processed_prod['prediction'] = model.predict(raw_prod.copy())
processed_train['prediction'] = model.predict(raw_X_test.copy())

data_drift_report = Dashboard(tabs=[DataQualityTab(), DataDriftTab()])
data_drift_report.calculate(processed_train, processed_prod, column_mapping = column_mapping)
data_drift_report.save("../reports/data_quality_drift_processed.html")

In [44]:
from evidently.dashboard.tabs import CatTargetDriftTab

processed_prod[config.TARGET] = target_prod[config.TARGET]
processed_train[config.TARGET] = raw_y_test[config.TARGET]
data_and_target_drift_report = Dashboard(tabs=[DataDriftTab(), CatTargetDriftTab()])
data_and_target_drift_report.calculate(processed_train, processed_prod, column_mapping = column_mapping)
data_and_target_drift_report.save("../reports/data_n_concept_drift.html")

In [43]:
from evidently.dashboard.tabs import ClassificationPerformanceTab

processed_prod[config.TARGET] = target_prod[config.TARGET]
rf_model = joblib.load(config.BASE_DIR +"/models/RandomForest_Best.sav")
rf_prod = processed_prod.copy()
rf_prod['prediction'] = rf_model.predict(raw_prod.copy())

dashboard = Dashboard(tabs=[ClassificationPerformanceTab(verbose_level=1)])
dashboard.calculate(reference_data=processed_prod, current_data=rf_prod, column_mapping = column_mapping)
dashboard.save('../reports/regression_report.html')